In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self, input_channels:int, nodes:list, kernels:list, num_classes:int=10):
        """
        input channels is the number of channels in the input data (e.g., 1 for grayscale, 3 for RGB)
        nodes is a list of nodes per layer i.e. 128, 64, 32
 kernals is a list of the kernals used per layer
 number of classes is thenumber of outputs (the number of emotions to possibly classify)
 this creates the neural network with the specified layers and the kernals
        """
        super(CustomCNN, self).__init__()

        assert len(nodes) == len(kernels), "nodes and kernels must have the same length"

        layers = []
        in_channels = input_channels

        for out_channels, k in zip(nodes, kernels):
            conv = nn.Conv2d(in_channels, out_channels, kernel_size=k, padding=k//2)
            layers.append(conv)
            layers.append(nn.ReLU())
            layers.append(nn.MaxPool2d(2))  
            in_channels = out_channels

        self.conv_layers = nn.Sequential(*layers)

        
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(nodes[-1], num_classes)  

    def forward(self, x):
        x = self.conv_layers(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))  
        x = self.flatten(x)
        x = self.fc(x)
        return x


In [ ]:

model = CustomCNN(
    input_channels=3,
    nodes=[32, 64, 128],
    kernels=[3, 3, 5],
    num_classes=10
)

print(model)
